![Neptune + Sacred](https://neptune.ai/wp-content/uploads/2023/09/sacred.svg)

# Neptune + Sacred

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/sacred/notebooks/Neptune_Sacred.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/sacred/notebooks/Neptune_Sacred.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/common/org/sacred-integration/runs/details?viewId=standard-view&detailsTab=dashboard&dashboardId=Sacred-Dashboard-6741ab33-825c-4b25-8ebb-bb95c11ca3f4&shortId=SAC-11"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/integrations/sacred/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

This guide will show you how to:

* Initialize Neptune and create a `run`,
* Log Sacred experiment metrics and atrifacts using `NeptuneObserver()`.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
%pip install -U neptune[sacred] sacred torch torchvision

## Basic Example

### Import Libraries

In [ ]:
import neptune
import torch
import torch.nn as nn
import torch.optim as optim

from neptune.integrations.sacred import NeptuneObserver
from sacred import Experiment
from torchvision import datasets, transforms

In [ ]:
if torch.device("cuda:0"):
    torch.cuda.empty_cache()

### Start a run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in the public project [common/sacred-integration](https://app.neptune.ai/o/common/org/sacred-integration). **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
    tags="notebook",
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
neptune_run = neptune.init_run(
    project="common/sacred-integration", api_token=neptune.ANONYMOUS_API_TOKEN, tags="notebook"
)

### Add NeptuneObserver() to your sacred experiment's observers

By using NeptuneObserver(), the following is automatically logged to the Neptune app for you:
- Hyperparameters
- Loss
- Metrics


In [ ]:
ex = Experiment("image_classification", interactive=True)
ex.observers.append(NeptuneObserver(run=neptune_run))

Define Model

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, input_sz=32 * 32 * 3, n_classes=10):
        super(BaseModel, self).__init__()
        self.lin = nn.Linear(input_sz, n_classes)

    def forward(self, input):
        x = input.view(-1, 32 * 32 * 3)
        return self.lin(x)

In [ ]:
model = BaseModel()

Define your configuration/hyperparamenters

In [ ]:
@ex.config
def cfg():
    data_dir = "data/CIFAR10"
    data_tfms = {
        "train": transforms.Compose(
            [
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )
    }
    lr = 1e-2
    bs = 128
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Define your run

In [ ]:
@ex.main
def run(data_dir, data_tfms, lr, bs, device, _run):
    trainset = datasets.CIFAR10(data_dir, transform=data_tfms["train"], download=True)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=bs, shuffle=True, num_workers=0)
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    for i, (x, y) in enumerate(trainloader, 0):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model.forward(x)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, y)
        acc = (torch.sum(preds == y.data)) / len(x)

        # Log loss
        ex.log_scalar("training/batch/loss", loss)
        # Log accuracy
        ex.log_scalar("training/batch/acc", acc)

        loss.backward()
        optimizer.step()

    return {"final_loss": loss.item(), "final_acc": acc.cpu().item()}

### Run your experiment and explore metadata in the Neptune app

All metadata is logged automatically to Neptune. 

After running your script or notebook cell you will get a link similar to:
https://app.neptune.ai/o/common/org/sacred-integration/e/SAC-11
with common/sacred-integration replaced by your project, and SAC-1 replaced by your run.

Click on the link to open the run in Neptune and watch your model training live.

In [ ]:
ex.run()

## More Options

### Log Artifacts

Model architecture and weights

In [ ]:
model_fname = "model"
print(f"Saving model archictecture as {model_fname}.txt")
with open(f"{model_fname}_arch.txt", "w") as f:
    f.write(str(model))

In [ ]:
print(f"Saving model weights as {model_fname}.pth")
torch.save(model.state_dict(), f"./{model_fname}.pth")

In [ ]:
ex.add_artifact(filename=f"./{model_fname}_arch.txt", name=f"{model_fname}_arch")
ex.add_artifact(filename=f"./{model_fname}.pth", name=model_fname)

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
neptune_run.stop()

## Analyze logged metadata in the Neptune app

Go to the run link and explore the metadata that were logged to the run in Neptune.

Link should look like this: https://app.neptune.ai/o/common/org/sacred-integration/runs/details?viewId=standard-view&detailsTab=metadata&shortId=SAC-11